In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir("/content/drive/My Drive/ssd300_sleep_check")

In [0]:
# パッケージのimport
import os.path as osp
import random
import time
import glob

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [0]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cuda:0


## globが認識されないのでmake_datapath_list()をここに書く

In [0]:
# 学習、検証の画像データとアノテーションデータへのファイルパスリストを作成する


def make_datapath_list(rootpath):
    """
    データへのパスを格納したリストを作成する。

    Parameters
    ----------
    rootpath : str
        データフォルダへのパス

    Returns
    -------
    ret : train_img_list, train_anno_list, val_img_list, val_anno_list
        データへのパスを格納したリスト
    """

    # 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
    #imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')
    #annopath_template = osp.join(rootpath, 'Annotations', '%s.xml')

    # 訓練と検証、それぞれのファイルのID（ファイル名）を取得する
    #train_id_names = osp.join(rootpath + 'ImageSets/Main/train.txt')
    #val_id_names = osp.join(rootpath + 'ImageSets/Main/val.txt')

    # 訓練データの画像ファイルとアノテーションファイルへのパスリストを作成
    """
    train_img_list = list()
    train_anno_list = list()
    for line in open(train_id_names):
        file_id = line.strip()  # 空白スペースと改行を除去
        img_path = (imgpath_template % file_id)  # 画像のパス
        anno_path = (annopath_template % file_id)  # アノテーションのパス
        train_img_list.append(img_path)  # リストに追加
        train_anno_list.append(anno_path)  # リストに追加
    """
    path_to_train_data = 'train/sleep_check_train2-PascalVOC-export/'
    train_img_list = glob.glob(rootpath + path_to_train_data + 'JPEGImages'+"/*")
    train_anno_list = glob.glob(rootpath + path_to_train_data + 'Annotations'+"/*")
    
    # 検証データの画像ファイルとアノテーションファイルへのパスリストを作成
    """
    val_img_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()  # 空白スペースと改行を除去
        img_path = (imgpath_template % file_id)  # 画像のパス
        anno_path = (annopath_template % file_id)  # アノテーションのパス
        val_img_list.append(img_path)  # リストに追加
        val_anno_list.append(anno_path)  # リストに追加
    """
    path_to_val_data = 'val/sleep_check_val2-PascalVOC-export/'
    val_img_list = glob.glob(rootpath + path_to_val_data + 'JPEGImages'+"/*")
    val_anno_list = glob.glob(rootpath + path_to_val_data + 'Annotations'+"/*")
    
    return train_img_list, train_anno_list, val_img_list, val_anno_list


In [33]:
os.getcwd()
rootpath = os.getcwd()+"/data/annotation_data_2/"
rootpath

'/content/drive/My Drive/ssd300_sleep_check/data/annotation_data_2/'

In [34]:
# ファイルパスのリストを作成
rootpath = os.getcwd()+"/data/annotation_data_2/" #datasetを作るときのために絶対パスにしておく
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# 動作確認
print(train_img_list[10])
print(val_img_list[10])

/content/drive/My Drive/ssd300_sleep_check/data/annotation_data_2/train/sleep_check_train2-PascalVOC-export/JPEGImages/10588052_02.jpg
/content/drive/My Drive/ssd300_sleep_check/data/annotation_data_2/val/sleep_check_val2-PascalVOC-export/JPEGImages/10588087_01.jpg


# DatasetとDataLoaderを作成する

In [0]:
os.chdir('SSD300_transfer/')

In [0]:
from utils.ssd_model import  VOCDataset, DataTransform, Anno_xml2list, od_collate_fn#make_datapath_listを消去

# Datasetを作成
voc_classes = ['safe', 'caution']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

#train_dataset.__getitem__(1)



## batch_size を32→8→32

In [0]:
# DataLoaderを作成する
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# ネットワークモデルの作成

In [38]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数(読み込んで変)
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg, transfer_class_num=3)
net

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

## 学習済みデータの読み込み

In [39]:
state_dict = torch.load("./weights/ssd300_mAP_77.43_v2.pth")
net.load_state_dict(state_dict)

<All keys matched successfully>

### conf層を３クラスに対応するように取り換え

In [0]:
net.conf[0] = nn.Conv2d(512, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
net.conf[1] = nn.Conv2d(1024, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
net.conf[2] = nn.Conv2d(512, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
net.conf[3] = nn.Conv2d(256, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
net.conf[4] = nn.Conv2d(256, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
net.conf[5] = nn.Conv2d(256, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [41]:
net.conf

ModuleList(
  (0): Conv2d(512, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

## conf層の初期値を設定

In [42]:
# SSDの初期の重みを設定
# ssdのvgg部分に重みをロードする
#vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')#vgg部分
#net.vgg.load_state_dict(vgg_weights)

# ssdのその他のネットワークの重みはHeの初期値で初期化

#Heの初期値を与える関数
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)#Heの初期値
        if m.bias is not None:  # バイアス項がある場合は０
            nn.init.constant_(m.bias, 0.0)


# Heの初期値を適用
#net.extras.apply(weights_init)
#net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

print('ネットワーク設定完了：学習済みの重みをロードしました')


使用デバイス： cuda:0
ネットワーク設定完了：学習済みの重みをロードしました


# 損失関数と最適化手法を定義する

In [43]:
#conf層のみ学習
net.conf.parameters()

<generator object Module.parameters at 0x7f38aa42a5c8>

In [0]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD([{'params':net.vgg.parameters(), 'lr':1e-4},
                       {'params':net.extras.parameters(), 'lr':1e-4},
                       {'params':net.L2Norm.parameters(), 'lr':1e-4},
                       {'params':net.loc.parameters(), 'lr':5e-4},
                       {'params':net.conf.parameters(), 'lr':1e-3}],momentum=0.9, weight_decay=5e-4)


#optimizer = optim.SGD(net.conf.parameters(), lr=1e-3,momentum=0.9, weight_decay=5e-4) #ハイパラメータ！！！


# 学習・検証を実施する

In [0]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は10回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_data2_fine' +
                       str(epoch+1) + '.pth')


In [46]:
# 学習・検証を実行する
num_epochs= 100  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-------------
Epoch 1/100
-------------
（train）
-------------
epoch 1 || Epoch_TRAIN_Loss:55.9510 ||Epoch_VAL_Loss:0.0000
timer:  22.6657 sec.
-------------
Epoch 2/100
-------------
（train）
イテレーション 10 || Loss: 6.7683 || 10iter: 18.9661 sec.
-------------
epoch 2 || Epoch_TRAIN_Loss:38.2262 ||Epoch_VAL_Loss:0.0000
timer:  19.3332 sec.
-------------
Epoch 3/100
-------------
（train）
-------------
epoch 3 || Epoch_TRAIN_Loss:31.7941 ||Epoch_VAL_Loss:0.0000
timer:  17.9182 sec.
-------------
Epoch 4/100
-------------
（train）
イテレーション 20 || Loss: 5.7867 || 10iter: 20.1449 sec.
-------------
epoch 4 || Epoch_TRAIN_Loss:30.2841 ||Epoch_VAL_Loss:0.0000
timer:  20.5294 sec.
-------------
Epoch 5/100
-------------
（train）
-------------
epoch 5 || Epoch_TRAIN_Loss:28.1327 ||Epoch_VAL_Loss:0.0000
timer:  18.2682 sec.
-------------
Epoch 6/100
-------------
（train）
イテレーション 30 || Loss: 5.2557 || 10iter: 19.6566 sec.
-------------
epoch 6 || Epoch_TRAIN_Loss:27.0596 ||Epoch_VAL_Loss:0.

以上